In [ ]:
!pip install keras-resnet==0.2.0
!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import tensorflow as tf
import numpy as np
import os
import re
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense, GlobalAveragePooling2D
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from tensorflow.keras.applications import ResNet50
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg19 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import preprocess_input
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Set the data directory path

In [ ]:
data_dir = '/content/drive/MyDrive/Colab Notebooks/archive (1)/Dataset_BUSI_with_GT'

We define the batch size and image dimensions here

In [ ]:
batch_size = 32
img_height = 224
img_width = 224

Since we are working with images from three differ classes/directories, we will use an Image data generator

# Don't want the masked images

In [ ]:
def is_not_mask_file(filename):
    return not re.search(r"_mask\.png$", filename)

In [ ]:
train_data_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    zoom_range=0.1,
    horizontal_flip=True,
    validation_split=0.2
)

val_data_gen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_data = train_data_gen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True,
    subset='training',
)

val_data = val_data_gen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False,
    subset='validation',
)

# Filter out mask images from train_data
train_data.filenames = [filename for filename in train_data.filenames if is_not_mask_file(filename)]
train_data.samples = len(train_data.filenames)

# Filter out mask images from val_data
val_data.filenames = [filename for filename in val_data.filenames if is_not_mask_file(filename)]
val_data.samples = len(val_data.filenames)

Found 1263 images belonging to 3 classes.
Found 315 images belonging to 3 classes.


# MODEL 1: CNN

In [ ]:
# Build model
classifier = Sequential()
classifier.add(Conv2D(filters=16, kernel_size=3, activation='relu', padding='same', input_shape=(img_height, img_width, 3)))
classifier.add(MaxPooling2D())
classifier.add(Conv2D(filters=32, kernel_size=3, activation='relu', padding='same'))
classifier.add(MaxPooling2D())
classifier.add(Conv2D(filters=64, kernel_size=3, activation='relu', padding='same'))
classifier.add(MaxPooling2D())
classifier.add(Dropout(0.5))
classifier.add(Flatten())
classifier.add(Dense(units=128, activation='relu'))
classifier.add(Dense(units=train_data.num_classes, activation='softmax'))

# Compile/train our model
classifier.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
classifier.fit(
    train_data,
    epochs=10,
    validation_data=val_data
)

# evaluate model
evaluation = classifier.evaluate(val_data)
print("Accuracy: " + str(evaluation[1]))

# Generate predictions
pred = classifier.predict(val_data)
pred = np.argmax(pred, axis=-1)

# Get true labels
val_labels = val_data.classes

# Class labels
class_labels = list(val_data.class_indices.keys())

# Generate classification report
report = classification_report(val_labels, pred, target_names=class_labels)
print(report)

# Generate confusion matrix
cm = confusion_matrix(val_labels, pred)
print("Confusion matrix:")
print(cm)

Epoch 1/10
40/40 [==============================] - 227s 5s/step - loss: 0.8951 - accuracy: 0.5851 - val_loss: 0.7717 - val_accuracy: 0.7048
Epoch 2/10
40/40 [==============================] - 25s 616ms/step - loss: 0.7023 - accuracy: 0.6746 - val_loss: 0.6929 - val_accuracy: 0.7079
Epoch 3/10
40/40 [==============================] - 24s 600ms/step - loss: 0.6523 - accuracy: 0.7039 - val_loss: 0.6706 - val_accuracy: 0.7397
Epoch 4/10
40/40 [==============================] - 24s 596ms/step - loss: 0.6210 - accuracy: 0.7300 - val_loss: 0.6968 - val_accuracy: 0.7079
Epoch 5/10
40/40 [==============================] - 24s 594ms/step - loss: 0.5638 - accuracy: 0.7451 - val_loss: 0.7177 - val_accuracy: 0.7238
Epoch 6/10
40/40 [==============================] - 24s 599ms/step - loss: 0.5283 - accuracy: 0.7775 - val_loss: 0.7049 - val_accuracy: 0.7048
Epoch 7/10
40/40 [==============================] - 26s 650ms/step - loss: 0.5039 - accuracy: 0.7783 - val_loss: 0.7119 - val_accuracy: 0.7460
E

# MODEL 2: VGG 16

In [ ]:
# Load pre-trained VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom classification layers on top of VGG16
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(train_data.num_classes, activation='softmax')(x)

# Create the model
model_vgg16 = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model_vgg16.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
epochs = 10
history = model_vgg16.fit(train_data, epochs=epochs, validation_data=val_data)

# Evaluate the model
val_loss, val_accuracy = model_vgg16.evaluate(val_data)
print('Validation Loss:', val_loss)
print('Validation Accuracy:', val_accuracy)

58889256/58889256 [==============================] - 0s 0us/step
Epoch 1/10
40/40 [==============================] - 37s 773ms/step - loss: 0.8870 - accuracy: 0.5867 - val_loss: 0.8088 - val_accuracy: 0.6095
Epoch 2/10
40/40 [==============================] - 27s 671ms/step - loss: 0.6954 - accuracy: 0.7237 - val_loss: 0.6926 - val_accuracy: 0.7365
Epoch 3/10
40/40 [==============================] - 27s 663ms/step - loss: 0.5859 - accuracy: 0.7823 - val_loss: 0.6339 - val_accuracy: 0.7556
Epoch 4/10
40/40 [==============================] - 28s 701ms/step - loss: 0.5161 - accuracy: 0.8013 - val_loss: 0.6325 - val_accuracy: 0.7365
Epoch 5/10
40/40 [==============================] - 27s 669ms/step - loss: 0.4713 - accuracy: 0.8290 - val_loss: 0.5790 - val_accuracy: 0.7683
Epoch 6/10
40/40 [==============================] - 26s 663ms/step - loss: 0.4488 - accuracy: 0.8211 - val_loss: 0.5701 - val_accuracy: 0.7778
Epoch 7/10
40/40 [==============================] - 26s 663ms/step - loss: 0.

# MODEL 3: VGG 19

In [ ]:
# Load pre-trained VGG19 model
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom classification layers on top of VGG19
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(train_data.num_classes, activation='softmax')(x)

# Create the model
model_vgg19 = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model_vgg19.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
epochs = 10
history = model_vgg19.fit(train_data, epochs=epochs, validation_data=val_data)

# Evaluate the model
val_loss, val_accuracy = model_vgg19.evaluate(val_data)
print('Validation Loss:', val_loss)
print('Validation Accuracy:', val_accuracy)

80134624/80134624 [==============================] - 1s 0us/step
Epoch 1/10
40/40 [==============================] - 27s 607ms/step - loss: 0.8529 - accuracy: 0.6049 - val_loss: 0.7566 - val_accuracy: 0.7365
Epoch 2/10
40/40 [==============================] - 25s 615ms/step - loss: 0.6573 - accuracy: 0.7522 - val_loss: 0.6602 - val_accuracy: 0.7429
Epoch 3/10
40/40 [==============================] - 24s 603ms/step - loss: 0.5582 - accuracy: 0.7941 - val_loss: 0.5999 - val_accuracy: 0.7714
Epoch 4/10
40/40 [==============================] - 25s 629ms/step - loss: 0.5134 - accuracy: 0.8076 - val_loss: 0.5567 - val_accuracy: 0.7810
Epoch 5/10
40/40 [==============================] - 25s 630ms/step - loss: 0.4699 - accuracy: 0.8282 - val_loss: 0.5402 - val_accuracy: 0.7873
Epoch 6/10
40/40 [==============================] - 27s 669ms/step - loss: 0.4492 - accuracy: 0.8179 - val_loss: 0.5193 - val_accuracy: 0.7937
Epoch 7/10
40/40 [==============================] - 24s 611ms/step - loss: 0.

# MODEL 4: RESNET-50 MODEL

In [ ]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom classification layers on top of ResNet50
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(train_data.num_classes, activation='softmax')(x)

# Create the model
model_resnet = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model_resnet.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
epochs = 10
history = model_resnet.fit(train_data, epochs=epochs, validation_data=val_data)

# Evaluate the model
val_loss, val_accuracy = model_resnet.evaluate(val_data)
print('Validation Loss:', val_loss)
print('Validation Accuracy:', val_accuracy)

94765736/94765736 [==============================] - 1s 0us/step
Epoch 1/10
40/40 [==============================] - 33s 690ms/step - loss: 0.9282 - accuracy: 0.5740 - val_loss: 0.8449 - val_accuracy: 0.5873
Epoch 2/10
40/40 [==============================] - 24s 597ms/step - loss: 0.8237 - accuracy: 0.6390 - val_loss: 0.7947 - val_accuracy: 0.6222
Epoch 3/10
40/40 [==============================] - 26s 663ms/step - loss: 0.7600 - accuracy: 0.6690 - val_loss: 0.8048 - val_accuracy: 0.6190
Epoch 4/10
40/40 [==============================] - 24s 613ms/step - loss: 0.7367 - accuracy: 0.6920 - val_loss: 0.7240 - val_accuracy: 0.7270
Epoch 5/10
40/40 [==============================] - 24s 592ms/step - loss: 0.6987 - accuracy: 0.7102 - val_loss: 0.7996 - val_accuracy: 0.6317
Epoch 6/10
40/40 [==============================] - 24s 602ms/step - loss: 0.6887 - accuracy: 0.7047 - val_loss: 0.7327 - val_accuracy: 0.6762
Epoch 7/10
40/40 [==============================] - 25s 637ms/step - loss: 0.

# Creating a confusion matrix

In [ ]:
from sklearn.metrics import confusion_matrix

pred = classifier.predict(val_data)
pred = np.argmax(pred, axis=-1)
true_labels = np.concatenate(val_labels)
cm = confusion_matrix(true_labels, pred)

10/10 [==============================] - 2s 223ms/step


ValueError: ignored

# Saving the models

In [ ]:
classifier.save("CNN.h5")
model_vgg16.save("VGG16.h5")
model_vgg19.save("VGG19.h5")
model_resnet.save("Resnet-50.h5")